In [1]:
import os, time, json, math
from pathlib import Path
import torch

REPORT = {"phase": []}

try:
    import torch
except Exception as e:
    print("[FAIL] torch import 실패:", e)
    raise SystemExit

if not torch.cuda.is_available():
    print("[FAIL] CUDA 사용 불가 — GPU가 보이지 않습니다.")
    raise SystemExit

# ---------- 파라미터(필요시만 바꾸세요) ----------
FRACTION = float(os.getenv("TORCH_PER_PROCESS_FRACTION", "0.45"))  # 각자 0.40~0.50 권장
ALLOC_TARGET_RATIO = 0.15   # 전체 VRAM의 15% 정도 텐서로 점진 할당(보수적)
BLOCK_MB = 256              # 한 번에 할당하는 블록 크기(MB). 너무 크게 잡으면 급격 OOM 가능
BATCH_START = 16            # 소형 합성 모델 배치 램프 시작값
BATCH_MAX = 128             # 램프 상한
IMG_SIZE = 224              # 224x224 합성 이미지
# ----------------------------------------------------

dev = torch.device("cuda")
idx = torch.cuda.current_device()
props = torch.cuda.get_device_properties(idx)
total = props.total_memory
total_gb = total / (1024**3)
name = torch.cuda.get_device_name(idx)

# (1) 소프트 상한(권고) 적용 — "하드 리밋"은 아니지만 충돌 완화에 도움
try:
    torch.cuda.set_per_process_memory_fraction(FRACTION, device=idx)
    REPORT["phase"].append(f"soft_cap_set:{FRACTION}")
except Exception as e:
    REPORT["phase"].append(f"soft_cap_failed:{e}")

# (2) 기본 정보 출력
print(f"[INFO] GPU: {name} | total ~{total_gb:.1f} GiB | use fraction≈{FRACTION}")

# (3) 소량 워밍업
torch.cuda.empty_cache(); torch.cuda.synchronize()
_ = torch.empty((1024,1024), device=dev); torch.cuda.synchronize()
del _; torch.cuda.empty_cache(); torch.cuda.synchronize()

# (4) 점진 텐서 할당 테스트 — 목표치까지 BLOCK 단위로 할당
target_bytes = int(total * ALLOC_TARGET_RATIO)  # e.g. 15%
block_bytes  = BLOCK_MB * 1024**2
alloc_list = []
allocated = 0
oom_during_tensor = False

print(f"[TEST] Tensor allocation ramp: target ≈ {target_bytes/1024**3:.2f} GiB "
      f"(blocks of {BLOCK_MB} MB)")
try:
    while allocated + block_bytes <= target_bytes:
        # float32 기준으로 개수 계산
        n = block_bytes // 4
        t = torch.empty(n, dtype=torch.float32, device=dev)
        alloc_list.append(t)
        allocated += block_bytes
        if len(alloc_list) % 8 == 0:
            torch.cuda.synchronize()
            cur = torch.cuda.memory_allocated(idx) / (1024**3)
            print(f"  - allocated ~{cur:.2f} GiB", end="\r")
    torch.cuda.synchronize()
    cur = torch.cuda.memory_allocated(idx) / (1024**3)
    print(f"  - allocated ~{cur:.2f} GiB (done)")
except RuntimeError as e:
    if "out of memory" in str(e).lower():
        oom_during_tensor = True
        print("\n[WARN] OOM during tensor ramp — 동시 사용 충돌 가능성")
    else:
        raise
finally:
    for t in alloc_list: del t
    torch.cuda.empty_cache(); torch.cuda.synchronize()

REPORT["tensor_ramp_ok"] = not oom_during_tensor
REPORT["tensor_ramp_target_gb"] = round(target_bytes/(1024**3), 2)

# (5) 합성 소형 모델로 배치 램프(AMP 미사용; 일반적 worst-case에 가깝게)
#     모델이 작아서 속도 빨라요. 배치를 키워가며 OOM을 보는 테스트.
import torch.nn as nn
import torch.nn.functional as F

class TinyNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool  = nn.AdaptiveAvgPool2d(1)
        self.fc    = nn.Linear(128, num_classes)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(x).view(x.size(0), -1)
        return self.fc(x)

model = TinyNet().to(dev)
opt = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

def try_batch(bs):
    x = torch.randn(bs, 3, IMG_SIZE, IMG_SIZE, device=dev)
    y = torch.randint(0, 1000, (bs,), device=dev)
    out = model(x)
    loss = loss_fn(out, y)
    loss.backward()
    opt.zero_grad(set_to_none=True)
    # 메모리 회수
    del x, y, out, loss
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

max_safe_bs = 0
oom_during_model = False
bs = BATCH_START
print(f"[TEST] Model batch ramp: start={BATCH_START}, up to {BATCH_MAX} (no AMP)")
while bs <= BATCH_MAX:
    try:
        try_batch(bs)
        max_safe_bs = bs
        print(f"  - OK @ batch={bs}", end="\r")
        bs *= 2 if bs < 32 else bs + 16  # 초반엔 빠르게, 이후엔 완만히 증가
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            oom_during_model = True
            print(f"\n[WARN] OOM @ batch={bs}")
            break
        else:
            raise

REPORT["model_ramp_ok"] = not oom_during_model
REPORT["model_max_safe_bs"] = int(max_safe_bs)

# (6) 요약 리포트
summary = {
    "gpu_name": name,
    "total_gb": round(total_gb, 2),
    "fraction_hint": FRACTION,
    "tensor_ramp_ok": REPORT["tensor_ramp_ok"],
    "tensor_ramp_target_gb": REPORT["tensor_ramp_target_gb"],
    "model_ramp_ok": REPORT["model_ramp_ok"],
    "model_max_safe_bs": REPORT["model_max_safe_bs"],
}
print("\n===== OOM SANITY SUMMARY =====")
print(json.dumps(summary, indent=2))

# (7) 간단 판정
if summary["tensor_ramp_ok"] and summary["model_ramp_ok"]:
    print("\n[PASS] 동시 실행 환경에서 보수적 수준에서는 OOM 없이 동작했습니다.")
else:
    print("\n[ATTN] 동시 실행 시 OOM 징후가 있습니다.")
    print(" - 배치 크기를 낮추거나 AMP 사용, 또는 다른 사용자의 부하와 시간 분리(순차 실행)를 권장합니다.")


[INFO] GPU: Tesla V100-SXM2-32GB | total ~31.7 GiB | use fraction≈0.45
[TEST] Tensor allocation ramp: target ≈ 4.76 GiB (blocks of 256 MB)
  - allocated ~4.75 GiB (done)
[TEST] Model batch ramp: start=16, up to 128 (no AMP)
  - OK @ batch=32
===== OOM SANITY SUMMARY =====
{
  "gpu_name": "Tesla V100-SXM2-32GB",
  "total_gb": 31.73,
  "fraction_hint": 0.45,
  "tensor_ramp_ok": true,
  "tensor_ramp_target_gb": 4.76,
  "model_ramp_ok": true,
  "model_max_safe_bs": 32
}

[PASS] 동시 실행 환경에서 보수적 수준에서는 OOM 없이 동작했습니다.
